In [1]:
import torch
import torch.optim as optim
from src.pytorch_efficientnet import EfficientNetwork, augment, augmented_pred
from src.train_test_functions import train_model
import json
import numpy as np
from src.torch_dataset import ZootrDataset 
import os
import pandas as pd
from sklearn.metrics import roc_auc_score
import mlflow
from src.mlflow_functions import start_mlflow
from contextlib import nullcontext


## Set constants

In [2]:
BATCH_SIZE = 20
NUM_WORKERS = 0
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.1
EPOCHS = 10

In [3]:
with open('config/paths_dict.json') as handle:
    paths_dict = json.loads(handle.read())

ITEM_PATH = paths_dict['item_path']
NON_ITEM_PATH = paths_dict['non_item_path']

In [4]:
load_data_objects = False
save_data_objects = True

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Set up data loaders

In [6]:
if  load_data_objects:
    torch.load('Models/train_data.pth')
    torch.load('Models/valid_data.pth')
else:
    items_images = [ITEM_PATH + i for i in os.listdir(ITEM_PATH)]
    #Get all items
    non_items_images = [NON_ITEM_PATH + i for i in os.listdir(NON_ITEM_PATH)]
    #Get non items
    all_images = list(np.random.permutation(items_images + non_items_images))
    #Combine all images' paths and shuffle
    train_data = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augment)
    train_data_eval = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augmented_pred)
    valid_data = ZootrDataset(all_images[int(np.ceil(len(all_images)*0.8)):], transform_=augmented_pred)
    # Data Objects
    if save_data_objects:
        torch.save(train_data, 'Models/train_data.pth')
        torch.save(valid_data, 'Models/valid_data.pth')

In [7]:
data = {'train': all_images[0:int(np.ceil(len(all_images)*0.7))], 
        'val': all_images[int(np.ceil(len(all_images)*0.7)):int(np.ceil(len(all_images)*0.8))], 
        'test': all_images[int(np.ceil(len(all_images)*0.8)):]}
transformations = {'train': augment, 'val': augmented_pred, 'test': augmented_pred}
dataset_sizes = {'train': int(np.ceil(len(all_images)*0.7)), 'val': int(np.ceil(len(all_images)*0.8)) - int(np.ceil(len(all_images)*0.7)), 
                 'test': len(all_images) - int(np.ceil(len(all_images)*0.8))}

datasets = {x: ZootrDataset(data[x], transform_=transformations[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, drop_last=True, shuffle=False) for x in ['train', 'val', 'test']}

dics = {
    'dataloaders': dataloaders,
    'dataset_sizes': dataset_sizes
}

## Model

In [8]:
model = EfficientNetwork(output_size=2, b1=False, b2=True)
# Criterion
criterion = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max',
                                                       patience=1, factor=0.4)

Loaded pretrained weights for efficientnet-b2


## Train

In [9]:
start_mlflow('zootr_img_id')

In [10]:
SAVE_RUN = True
NR_EPOCHS=3


In [ ]:
with mlflow.start_run() if SAVE_RUN else nullcontext() as run:

  mdl, loss_dict_collection = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=NR_EPOCHS)
  test_loader = dics['dataloaders']['test']

  all_preds = torch.empty(0, 2)
  all_labels = []
  for imgs, label in test_loader:
      imgs_pred = mdl(imgs)

      all_preds = torch.cat((all_preds, imgs_pred))
      all_labels += label
  labels = [i.item() for i in all_labels]

  preds_prob = torch.softmax(all_preds, dim=1)
  preds_prob = pd.DataFrame(preds_prob.detach().numpy())
  preds_prob.columns = ['False', 'True']
  preds_prob['Label'] = labels

  test_loss = criterion(all_preds, torch.tensor(labels)).item()
  test_roc = roc_auc_score(preds_prob['Label'], preds_prob['True'])

  if SAVE_RUN:
    params = {
       "num_epochs": NR_EPOCHS,
        "scheduler": scheduler.__dict__,
        "optimizer": optimizer
        }
    mlflow.log_params(params)
    
    mlflow.log_dict(loss_dict_collection, 'loss_dict_collection.json')
    torch.save(model, 'base_model.pt')
    mlflow.log_artifact('base_model.pt')

    mlflow.log_metrics({"test_loss": test_loss, "test_roc": test_roc})

Epoch 1


100%|██████████| 10/10 [00:06<00:00,  1.51it/s]


Train loss: 0.4128363635841533 Val loss: 0.23871323768643365
Epoch 2


100%|██████████| 10/10 [00:07<00:00,  1.31it/s]


Train loss: 0.1423715013796985 Val loss: 0.1424512375092161
Epoch 3


100%|██████████| 10/10 [00:06<00:00,  1.45it/s]


Train loss: 0.07764994536874702 Val loss: 0.12098712603683057
Training complete in 3m 10s
Best val loss: 0.077650
🏃 View run gaudy-crab-45 at: http://localhost:5000/#/experiments/873592191447312121/runs/4d348167cac344c19cf1e05749792343
🧪 View experiment at: http://localhost:5000/#/experiments/873592191447312121


In [12]:
1/0

ZeroDivisionError: division by zero

In [13]:
params = {
"num_epochs": NR_EPOCHS,
"scheduler": scheduler.__dict__,
"optimizer": optimizer
}
mlflow.log_params(params)

mlflow.log_dict(loss_dict_collection, 'loss_dict_collection.json')
torch.save(model, 'base_model.pt')
mlflow.log_artifact('base_model.pt')

mlflow.log_metrics({"test_loss": test_loss, "test_roc": test_roc})